<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# Appendix A: Introduction to PyTorch (Part 2)
# 附录 A: PyTorch 入门（第 2 部分）

## A.9 Optimizing training performance with GPUs
## A.9 使用 GPU 优化训练性能

### A.9.1 PyTorch computations on GPU devices
### A.9.1 在 GPU 设备上进行 PyTorch 计算

In [1]:
# 导入 PyTorch 库
import torch

# 打印 PyTorch 版本号
print(torch.__version__)

2.4.0+cu121


In [2]:
# 检查是否有可用的 CUDA GPU 设备
print(torch.cuda.is_available())

True


In [3]:
# 创建第一个张量，包含浮点数 1,2,3
tensor_1 = torch.tensor([1., 2., 3.])
# 创建第二个张量，包含浮点数 4,5,6 
tensor_2 = torch.tensor([4., 5., 6.])

# 打印两个张量相加的结果
print(tensor_1 + tensor_2)

tensor([5., 7., 9.])


In [4]:
# 将 tensor_1 移动到 CUDA GPU 设备上
tensor_1 = tensor_1.to("cuda")
# 将 tensor_2 移动到 CUDA GPU 设备上
tensor_2 = tensor_2.to("cuda")

# 打印在 GPU 上相加的结果
print(tensor_1 + tensor_2)

tensor([5., 7., 9.], device='cuda:0')


In [5]:
# 将 tensor_1 移动到 CPU 设备上
tensor_1 = tensor_1.to("cpu")
# 尝试将一个在 CPU 上的张量和一个在 GPU 上的张量相加
# 这会导致错误，因为不能在不同设备上的张量之间进行运算
print(tensor_1 + tensor_2)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

### A.9.2 Single-GPU training
### A.9.2 单GPU训练

In [6]:
# 创建训练数据特征矩阵，包含5个样本，每个样本有2个特征
X_train = torch.tensor([
    [-1.2, 3.1],   # 第1个样本：特征1为-1.2，特征2为3.1
    [-0.9, 2.9],   # 第2个样本：特征1为-0.9，特征2为2.9 
    [-0.5, 2.6],   # 第3个样本：特征1为-0.5，特征2为2.6
    [2.3, -1.1],   # 第4个样本：特征1为2.3，特征2为-1.1
    [2.7, -1.5]    # 第5个样本：特征1为2.7，特征2为-1.5
])

# 创建训练数据标签向量，0表示第一类，1表示第二类
y_train = torch.tensor([0, 0, 0, 1, 1])

# 创建测试数据特征矩阵，包含2个样本，每个样本有2个特征
X_test = torch.tensor([
    [-0.8, 2.8],   # 第1个测试样本：特征1为-0.8，特征2为2.8
    [2.6, -1.6],   # 第2个测试样本：特征1为2.6，特征2为-1.6
])

# 创建测试数据标签向量
y_test = torch.tensor([0, 1])

In [7]:
# 导入PyTorch的Dataset类，用于创建自定义数据集
from torch.utils.data import Dataset


# 创建一个玩具数据集类，继承自Dataset
class ToyDataset(Dataset):
    def __init__(self, X, y):
        # 初始化函数，接收特征矩阵X和标签向量y
        self.features = X  # 存储特征数据
        self.labels = y    # 存储标签数据

    def __getitem__(self, index):
        # 获取单个样本的方法，返回对应索引的特征和标签
        one_x = self.features[index]  # 获取一个样本的特征
        one_y = self.labels[index]    # 获取一个样本的标签
        return one_x, one_y

    def __len__(self):
        # 返回数据集的样本数量
        return self.labels.shape[0]

# 创建训练数据集实例
train_ds = ToyDataset(X_train, y_train)
# 创建测试数据集实例
test_ds = ToyDataset(X_test, y_test)

In [8]:
# 导入PyTorch的DataLoader类，用于创建数据加载器
from torch.utils.data import DataLoader

# 设置随机种子以确保结果可重复
torch.manual_seed(123)

# 创建训练数据加载器
train_loader = DataLoader(
    dataset=train_ds,      # 使用之前创建的训练数据集
    batch_size=2,          # 每个批次包含2个样本
    shuffle=True,          # 随机打乱数据
    num_workers=1,         # 使用1个工作进程加载数据
    drop_last=True         # 丢弃最后一个不完整的批次
)

# 创建测试数据加载器
test_loader = DataLoader(
    dataset=test_ds,       # 使用之前创建的测试数据集
    batch_size=2,          # 每个批次包含2个样本
    shuffle=False,         # 不打乱测试数据的顺序
    num_workers=1          # 使用1个工作进程加载数据
)

In [9]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        # 继承父类的初始化方法
        super().__init__()

        # 定义神经网络的层结构
        self.layers = torch.nn.Sequential(

            # 第一个隐藏层
            # 输入维度为num_inputs，输出维度为30
            torch.nn.Linear(num_inputs, 30),
            # ReLU激活函数
            torch.nn.ReLU(),

            # 第二个隐藏层
            # 输入维度为30，输出维度为20
            torch.nn.Linear(30, 20),
            # ReLU激活函数
            torch.nn.ReLU(),

            # 输出层
            # 输入维度为20，输出维度为num_outputs
            torch.nn.Linear(20, num_outputs),
        )

    def forward(self, x):
        # 前向传播函数
        # 输入x通过网络层得到logits输出
        logits = self.layers(x)
        return logits

In [10]:
# 导入PyTorch的函数式模块，用于损失函数等操作
import torch.nn.functional as F


# 设置随机种子以确保结果可重现
torch.manual_seed(123)
# 创建神经网络模型实例，输入维度为2，输出维度为2
model = NeuralNetwork(num_inputs=2, num_outputs=2)

# 检测是否有可用的GPU，如果有则使用GPU，否则使用CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # NEW
# 将模型移动到指定设备(GPU/CPU)
model = model.to(device) # NEW

# 创建SGD优化器，学习率设为0.5
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

# 设置训练轮数为3
num_epochs = 3

# 开始训练循环，遍历每个epoch
for epoch in range(num_epochs):

    # 将模型设置为训练模式
    model.train()
    # 遍历训练数据加载器中的每个批次
    for batch_idx, (features, labels) in enumerate(train_loader):

        # 将特征和标签数据移动到指定设备
        features, labels = features.to(device), labels.to(device) # NEW
        # 前向传播，获得模型预测结果
        logits = model(features)
        # 计算交叉熵损失
        loss = F.cross_entropy(logits, labels) # Loss function

        # 清零梯度
        optimizer.zero_grad()
        # 反向传播计算梯度
        loss.backward()
        # 更新模型参数
        optimizer.step()

        # 打印训练进度和损失值
        print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
              f" | Batch {batch_idx:03d}/{len(train_loader):03d}"
              f" | Train/Val Loss: {loss:.2f}")

    # 将模型设置为评估模式
    model.eval()
    # 可选的模型评估步骤

Epoch: 001/003 | Batch 000/002 | Train/Val Loss: 0.75
Epoch: 001/003 | Batch 001/002 | Train/Val Loss: 0.65
Epoch: 002/003 | Batch 000/002 | Train/Val Loss: 0.44
Epoch: 002/003 | Batch 001/002 | Train/Val Loss: 0.13
Epoch: 003/003 | Batch 000/002 | Train/Val Loss: 0.03
Epoch: 003/003 | Batch 001/002 | Train/Val Loss: 0.00


In [11]:
def compute_accuracy(model, dataloader, device):
    """计算模型在给定数据加载器上的准确率
    
    参数:
        model: 神经网络模型
        dataloader: 数据加载器
        device: 计算设备(GPU/CPU)
        
    返回:
        float: 准确率(0-1之间的浮点数)
    """
    # 将模型设置为评估模式
    model = model.eval()
    # 初始化正确预测的样本数
    correct = 0.0
    # 初始化总样本数
    total_examples = 0

    # 遍历数据加载器中的每个批次
    for idx, (features, labels) in enumerate(dataloader):
        # 将特征和标签移动到指定设备
        features, labels = features.to(device), labels.to(device)

        # 在不计算梯度的情况下进行前向传播
        with torch.no_grad():
            logits = model(features)

        # 获取每个样本的预测类别(概率最大的类别)
        predictions = torch.argmax(logits, dim=1)
        # 比较预测结果与真实标签
        compare = labels == predictions
        # 累加正确预测的样本数
        correct += torch.sum(compare)
        # 累加总样本数
        total_examples += len(compare)

    # 返回准确率
    return (correct / total_examples).item()

In [12]:
# 计算模型在训练集上的准确率
compute_accuracy(model, train_loader, device=device)

1.0

In [13]:
# 计算模型在测试集上的准确率
compute_accuracy(model, test_loader, device=device)

1.0

### A.9.3 Training with multiple GPUs
### A.9.3 使用多个GPU进行训练

See [DDP-script.py](DDP-script.py)

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/appendix-a_compressed/12.webp" width="600px">
<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/appendix-a_compressed/13.webp" width="600px">